# RAG 시스템 - 뷰티 코칭 AI (Hybrid: HuggingFace + OpenAI)

이 노트북은 PDF 문서를 기반으로 한 RAG(Retrieval-Augmented Generation) 시스템입니다.

## 하이브리드 접근 (비용 효율적!) ⭐
- **임베딩**: HuggingFace multilingual-e5-large (무료, GPU/CPU, 다국어 지원)
- **LLM**: OpenAI GPT-4o-mini (유료, 고품질, 비전 지원)

## 주요 기능
- PDF 문서 로드 및 전처리
- HuggingFace 다국어 임베딩 및 Chroma DB 저장 (무료!)
- 한글 쿼리로 영어 논문도 검색 가능
- OpenAI GPT-4o-mini 비전 모델을 사용한 이미지+텍스트 질의응답

## 1. 라이브러리 Import

In [1]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import re
import json
import time
import os
import base64
from pathlib import Path

load_dotenv()

# OpenAI API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
else:
    print("✅ OpenAI API 키가 설정되었습니다.")

✅ OpenAI API 키가 설정되었습니다.


## 2. 유틸리티 함수 정의

In [2]:
def clean_text(text):
    """텍스트 공백 정리"""
    text = re.sub(r' {2,}', ' ', text)
    text = text.replace('\t', ' ')
    text = re.sub(r'\n{3,}', '\n\n', text)
    lines = [line.strip() for line in text.split('\n')]
    text = '\n'.join(lines)
    lines = [line for line in text.split('\n') if line]
    text = '\n'.join(lines)
    return text.strip()

def load_documents():
    """PDF 문서 로드"""
    loader = DirectoryLoader(
        "data/",
        glob="**/*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    for doc in documents:
        doc.page_content = clean_text(doc.page_content)
    return documents

def chunk_doc(documents, chunk_size=500, chunk_overlap=50):
    """문서를 청크로 분할"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],
        keep_separator=True
    )
    splits = text_splitter.split_documents(documents)
    return splits

def save_documents(documents, output_path):
    """문서를 JSONL 형식으로 저장"""
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, page in enumerate(documents):
            data = {
                'page': i + 1,
                'text': page.page_content,
                'metadata': {
                    'source': 'sample.pdf',
                    'page': page.metadata.get('page', i)
                }
            }
            f.write(json.dumps(data, ensure_ascii=False) + '\n')

def encode_image(image_path):
    """이미지를 base64로 인코딩"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

print("✅ 유틸리티 함수 정의 완료")

✅ 유틸리티 함수 정의 완료


## 3. 문서 로드 및 전처리

**이 셀은 한 번만 실행하세요!**

In [3]:
print("문서 로드 중...")
documents = load_documents()
print(f"✅ 총 {len(documents)}개의 문서 로드 완료")

print("\n문서 청킹 중...")
chunked_documents = chunk_doc(documents)
print(f"✅ 총 {len(chunked_documents)}개의 청크 생성 완료")

print("\n문서 저장 중...")
save_documents(chunked_documents, 'tmp.jsonl')
print("✅ tmp.jsonl 저장 완료")

문서 로드 중...
✅ 총 1164개의 문서 로드 완료

문서 청킹 중...
✅ 총 8011개의 청크 생성 완료

문서 저장 중...
✅ tmp.jsonl 저장 완료


## 4. 벡터 임베딩 및 저장 (HuggingFace multilingual-e5-large)

**이 셀도 한 번만 실행하세요! 이미 임베딩이 있다면 5번 셀로 바로 가세요.**

**HuggingFace 다국어 모델 사용 - 무료!** ⭐
**한글 쿼리로 영어 논문도 검색 가능!**

In [4]:
print("=" * 50)
print("HuggingFace multilingual-e5-large 임베딩 시작...")
print(f"청크 개수: {len(chunked_documents)}개")
print("⭐ HuggingFace 다국어 모델 사용 - 무료!")
print("⭐ 한글 쿼리로 영어 논문도 검색 가능!")
print("=" * 50)

embedding_start = time.time()

# HuggingFace multilingual-e5-large 임베딩 모델 사용
# 다국어 지원으로 한글 쿼리로 영어 문서 검색 가능
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    # model_kwargs={'device': 'cuda'},  # GPU 사용 (없으면 'cpu'로 변경)
    model_kwargs={'device': 'mps'},  # GPU 사용 (없으면 'cpu'로 변경)
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ 임베딩 모델 로드 완료")
print("📥 벡터 임베딩 생성 중... (시간이 다소 걸릴 수 있습니다)")

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
    persist_directory="./chroma_db_e5"
)

embedding_end = time.time()
print(f"✅ HuggingFace multilingual-e5-large 임베딩 완료: {embedding_end - embedding_start:.2f}초")
print("=" * 50)

HuggingFace multilingual-e5-large 임베딩 시작...
청크 개수: 8011개
⭐ HuggingFace 다국어 모델 사용 - 무료!
⭐ 한글 쿼리로 영어 논문도 검색 가능!
✅ 임베딩 모델 로드 완료
📥 벡터 임베딩 생성 중... (시간이 다소 걸릴 수 있습니다)
✅ HuggingFace multilingual-e5-large 임베딩 완료: 2113.66초


## 5. 기존 벡터스토어 로드 (HuggingFace multilingual-e5-large)

**임베딩을 다시 하지 않고 기존 것을 사용하려면 이 셀을 실행하세요!**

**HuggingFace 다국어 모델 - 무료!** ⭐

In [5]:
if os.path.exists("./chroma_db_e5"):
    print("기존 HuggingFace multilingual-e5-large 벡터스토어 로드 중...")
    
    embeddings = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-large",
        # model_kwargs={'device': 'cuda'},  # GPU 사용 (없으면 'cpu'로 변경)
        model_kwargs={'device': 'mps'},  # GPU 사용 (없으면 'cpu'로 변경)
        encode_kwargs={'normalize_embeddings': True}
    )
    
    vectorstore = Chroma(
        persist_directory="./chroma_db_e5",
        embedding_function=embeddings
    )
    print("✅ 벡터스토어 로드 완료")
else:
    print("❌ 벡터스토어가 없습니다. 4번 셀을 먼저 실행하세요.")

기존 HuggingFace multilingual-e5-large 벡터스토어 로드 중...
✅ 벡터스토어 로드 완료


/var/folders/hf/6g75lk2j3_n652n7198vpl1w0000gn/T/ipykernel_53735/835407575.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


## 6. RAG 체인 설정 (Hybrid: HuggingFace 임베딩 + OpenAI GPT-4o-mini)

**비용 효율적인 하이브리드 접근:**
- **임베딩**: HuggingFace multilingual-e5-large (무료, GPU/CPU, 다국어 지원)
- **LLM/Chat**: OpenAI GPT-4o-mini (유료, 고품질, 비전 지원)

In [6]:
# Retriever 설정
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 15}  # 중복 제거를 고려해서 많이 가져옴
)

# OpenAI GPT-4o-mini 모델 설정 (비전 지원)
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

print(f"✅ 사용 모델: gpt-4o-mini (비전 지원)")

# LCEL 방식으로 체인 구성
def deduplicate_docs(docs):
    """중복 문서 제거"""
    seen = set()
    unique_docs = []
    for doc in docs:
        # 문서 내용으로 중복 체크
        content_hash = hash(doc.page_content)
        if content_hash not in seen:
            seen.add(content_hash)
            unique_docs.append(doc)
    return unique_docs[:7]  # 최종적으로 7개만 반환

def format_docs(docs):
    """중복 제거 후 포맷팅"""
    unique_docs = deduplicate_docs(docs)
    return "\n\n".join(doc.page_content for doc in unique_docs)

def create_message_with_image(inputs):
    """이미지와 텍스트를 함께 메시지로 생성"""
    context = inputs["context"]
    question = inputs["question"]
    image_data = inputs.get("image")

    # 프롬프트 텍스트 구성 (한글 답변 강제)
    prompt_text = f"""당신은 다음 3명의 전문가가 통합된 고급 이미지·뷰티 코칭 AI입니다.
- 1. 헤어 스타일리스트: 헤어 형태, 볼륨, 질감, 스타일 방향성, 머릿결 관리 등 분석
- 2. 피부 관리 전문가: 피부 톤, 수분·유분 상태, 결, 잡티, 광택 등 관찰
- 3. 얼굴 윤곽 디자이너: 눈·코·입 비율, 턱선, 볼살, 이마 곡선, 대칭, 입체감 등 분석

[분석 방식 지침]
1. 사용자가 제공한 성별, 직업, 상황 설명을 기반으로 한다.
2. 관찰은 반드시 시각적·감각적 요소 중 최소 3가지 이상을 활용하고 디테일을 풍부하게 묘사하여, 독자가 장면을 그릴 수 있게 작성한다.
3. 부정적인 표현을 피하고 개선 가능성을 중심으로 서술한다.
4. 개선 팁은 오늘 실천 가능한 구체적 행동이나 셀프 케어 방법만 포함해야 하며, 추상적 조언은 금지된다.
5. 따뜻하고 격려적인 어조를 유지하며, 객관적이되 전문적으로 작성한다.
6. 전문 용어를 적절히 사용하되 이해하기 쉬운 표현으로 설명한다.

[중요: 질문 답변 우선 원칙]
- 사용자가 특정 질문(예: "염색 추천해줘", "헤어스타일 추천해줘")을 한 경우, 반드시 해당 질문에 대한 구체적인 답변을 먼저 제공해야 합니다.
- 질문과 관련된 카테고리(Hair/Skin/Contour)의 status에 질문에 대한 답변을 포함시키세요.
- 예시: "염색 추천해줘" → Hair.status에 "현재 [기존 헤어 상태]. 화사한 느낌을 위해 [구체적 염색 색상과 톤] 추천드립니다" 형식으로 작성

[논문 인용 원칙] ⭐ 중요 ⭐
- 참고 문서에 관련 논문 내용이 있으면 **반드시 직접 인용**하세요.
- 인용 시 따옴표("")를 사용하고, 구체적인 연구 결과, 수치, 전문 용어를 포함하세요.
- 예시: "연구에 따르면 '열펌 시술 시 180도 이상의 고온은 모발의 단백질 변성을 일으킬 수 있다'고 합니다"
- 예시: "논문에서는 '시스틴 결합이 부분적으로 절단되고 각각 수소원자와 결합되어 고분자의 시스테인을 만들게 된다'고 설명합니다"
- status와 improvement_tips 모두에서 논문 내용을 적극적으로 인용하여 전문성을 높이세요.

[분석 카테고리]
1. 헤어(Hair)
2. 피부(Skin)
3. 윤곽(Contour)

[참고 문서]
{context}

[사용자 질문]
{question}

[Output JSON Example]
{{{{
  "Appearance_Coaching": {{{{
    "Hair": {{{{
      "status": "헤어의 현재 상태에 대한 간결한 요약 설명. 만약 사용자가 헤어 관련 질문(염색, 스타일 등)을 했다면, 여기에 구체적인 추천을 포함하여 작성. 참고 문서의 논문 내용이 있다면 따옴표를 사용하여 직접 인용",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1 (논문 인용 포함 가능)",
        "오늘 실천 가능한 구체적인 행동 2 (논문 인용 포함 가능)"
      ]
    }}}},
    "Skin": {{{{
      "status": "피부의 현재 상태에 대한 간결한 요약 설명. 만약 사용자가 피부/메이크업 관련 질문을 했다면, 여기에 구체적인 추천을 포함하여 작성. 논문 내용 직접 인용",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1 (논문 인용 포함 가능)",
        "오늘 실천 가능한 구체적인 행동 2 (논문 인용 포함 가능)"
      ]
    }}}},
    "Contour": {{{{
      "status": "윤곽의 현재 상태에 대한 간결한 요약 설명. 논문 내용 직접 인용",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1 (논문 인용 포함 가능)",
        "오늘 실천 가능한 구체적인 행동 2 (논문 인용 포함 가능)"
      ]
    }}}}
  }}}}
}}}}

**중요 규칙:**
1. 답변은 반드시 한글로만 작성해야 합니다. 영어는 절대 사용하지 마세요.
2. 출력은 반드시 위 JSON 형식으로 하며, 개선 팁은 반드시 오늘 실천 가능한 구체적 행동으로 작성하세요.
3. 사용자 질문이 특정 주제(염색, 헤어스타일, 메이크업 등)에 대한 것이면, 해당 카테고리의 status에 질문에 대한 구체적인 답변을 우선적으로 포함하세요.
4. status는 시각적·감각적 요소를 3가지 이상 포함하되, 질문에 대한 답변을 먼저 작성하고 현재 상태 분석을 추가하세요.
5. **참고 문서에 논문 내용이 있으면 반드시 따옴표("")를 사용하여 직접 인용하고, 연구 결과나 전문 지식을 적극 활용하세요.**

답변:"""

    # 이미지가 있는 경우 멀티모달 메시지 생성
    if image_data:
        return [HumanMessage(
            content=[
                {"type": "text", "text": prompt_text},
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}
                }
            ]
        )]
    else:
        # 이미지가 없는 경우 텍스트만 전달
        return [HumanMessage(content=prompt_text)]

qa_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x["question"])),
        "question": lambda x: x["question"],
        "image": lambda x: x.get("image")
    }
    | RunnableLambda(create_message_with_image)
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인 설정 완료 (이미지 지원, 중복 제거 기능 추가)")

✅ 사용 모델: gpt-4o-mini (비전 지원)
✅ RAG 체인 설정 완료 (이미지 지원, 중복 제거 기능 추가)


In [7]:
test_query = "This picture is a Korean celebrity, so please evaluate whether people can objectively think that they are beautiful, and let me know what parts of your face can be improved."
retriever.invoke(test_query)

[Document(metadata={'author': 'Alice S. Liu', 'creationdate': '2023-10-31T18:02:59+05:30', 'page': 8, 'moddate': '2023-10-31T19:06:53+05:30', 'creator': 'Servigistics Arbortext Advanced Print Publisher 11.1.4546/W-x64', 'producer': 'PDFlib+PDI 9.0.7p3 (C++/Win64)', 'title': 'Using Artificial Intelligence to Quantify Sexual Dimorphism in Aesthetic Faces: Analysis of 100 Facial Points in 42 Caucasian Celebrities', 'page_label': '9', 'epsprocessor': 'PStill version 1.84.42', 'keywords': '', 'subject': 'DOI: 10.1093/asjof/ojad046; Aesthetic Surgery Journal Open Forum, 5, 0, 2023-6-5.; Abstract:  Background: Sexual dimorphism has been studied in the faces of average populations and worldwide celebrities; however, a focused analysis of attractive Caucasian faces has not been conducted. Objective: The study harnesses the power of artificial intelligence (AI) to efficiently analyze these facial patterns in attractive Caucasian male and female celebrities. Methods: Twenty-one male and 21 female

## 7. 질의응답 실행 (텍스트만)

**이미지 없이 텍스트만 사용하는 경우**

In [13]:
# 질문을 여기서 수정하세요
query = "면접을 볼 때 어떤 사진을 제출해야할지 감이 안잡히는데, 어떤 사진이 있으면 좋을까요?"

print("=" * 50)
print(f"질문: {query}")
print("=" * 50)
print("\nRAG 쿼리 시작... (OpenAI API 호출 중)")
invoke_start = time.time()

result = qa_chain.invoke({"question": query})

invoke_end = time.time()
print(f"✅ RAG 쿼리 완료: {invoke_end - invoke_start:.2f}초")
print("=" * 50)

print("\n📝 답변:")
print(result)

질문: 면접을 볼 때 어떤 사진을 제출해야할지 감이 안잡히는데, 어떤 사진이 있으면 좋을까요?

RAG 쿼리 시작... (OpenAI API 호출 중)
✅ RAG 쿼리 완료: 23.05초

📝 답변:
```json
{
  "Appearance_Coaching": {
    "Hair": {
      "status": "현재 머리카락은 건강하고 자연스러운 윤기를 띠고 있으며, 길이는 어깨에 닿는 정도입니다. 볼륨감이 적당하고, 텍스처는 부드러우며 손질하기 용이합니다. 면접 사진에서는 깔끔하면서도 세련된 느낌을 주는 스타일이 좋습니다. 예를 들어, 이마를 드러내는 정갈한 업스타일이나 부드러운 웨이브를 추천드립니다. 연구에 따르면 '긴 머리보다 짧은 스타일이 더 전문적인 이미지를 줄 수 있다'고 합니다.",
      "improvement_tips": [
        "머리카락의 건강을 유지하기 위해, 매일 아침 샴푸 후 에센스를 사용하여 수분을 보충해 주세요. 이는 모발의 질감을 개선하는 데 도움이 됩니다.",
        "매주 한 번은 영양이 풍부한 헤어 마스크를 적용하여 모발에 깊은 보습을 주세요. 연구에 따르면 '정기적인 모발 관리가 모발 손상을 예방하는 데 효과적'이라는 결과가 있습니다."
      ]
    },
    "Skin": {
      "status": "피부는 밝고 고른 톤을 띄고 있으며, 수분감이 적당하지만 약간의 유분이 느껴집니다. 결은 매끄러우며, 잡티는 없지만 약간의 광택이 보입니다. 면접 사진에서는 자연스러운 메이크업으로 피부 결을 강조하며, 생기를 주는 것이 좋습니다. '피부의 수분과 유분 균형을 맞추는 것이 건강한 피부를 만드는 핵심'이라고 보고된 바 있습니다.",
      "improvement_tips": [
        "하루에 두 번, 알로에 베라 젤을 사용하여 피부에 수분을 공급해 보세요. 이는 피부의 수분을 유지하는 데 도움을 줍니다.",
        "주 2회 각질 제거를 통해 피부 결을 매끄럽게 유

## 8. 질의응답 실행 (이미지 포함)

**이미지와 함께 질문하는 경우**

**주의: GPT-4o 비전 모델은 비용이 더 많이 발생합니다!**

In [12]:
# 이미지 경로와 질문을 수정하세요
image_path = "image/sanghyol_여권.jpg"
query = "면접을 볼 때 이 사진으로 이력서를 제출하려고 하는데, 면접관들이 보기에 이 사진이 전문적이고 신뢰감을 줄 수 있을까요?"

if Path(image_path).exists():
    print("=" * 50)
    print(f"이미지: {image_path}")
    print(f"질문: {query}")
    print("=" * 50)
    print("\n이미지 인코딩 중...")
    
    image_base64 = encode_image(image_path)
    print("✅ 이미지 인코딩 완료")
    
    print("\nRAG 쿼리 시작... (OpenAI GPT-4o Vision API 호출 중)")
    invoke_start = time.time()
    
    result = qa_chain.invoke({
        "question": query,
        "image": image_base64
    })
    
    invoke_end = time.time()
    print(f"✅ RAG 쿼리 완료: {invoke_end - invoke_start:.2f}초")
    print("=" * 50)
    
    print("\n📝 답변:")
    print(result)
else:
    print(f"❌ 이미지 파일을 찾을 수 없습니다: {image_path}")

이미지: image/sanghyol_여권.jpg
질문: 면접을 볼 때 이 사진으로 이력서를 제출하려고 하는데, 면접관들이 보기에 이 사진이 전문적이고 신뢰감을 줄 수 있을까요?

이미지 인코딩 중...
✅ 이미지 인코딩 완료

RAG 쿼리 시작... (OpenAI GPT-4o Vision API 호출 중)
✅ RAG 쿼리 완료: 16.12초

📝 답변:
{
  "Appearance_Coaching": {
    "Hair": {
      "status": "현재 헤어는 깔끔하게 정돈된 모습으로, 볼륨감이 적당하며 자연스러운 윤기가 느껴집니다. 앞머리가 뚜렷하게 잘라져 있어 세련된 인상을 주지만, 약간의 텍스처가 부족하여 더욱 생동감 있는 스타일로 개선할 수 있을 것 같습니다. 전문적인 이미지를 주기에 적합한 스타일입니다.",
      "improvement_tips": [
        "모발에 자연스러운 윤기를 주기 위해, 매일 아침 모발을 부드럽게 드라이한 후, 세럼이나 오일을 소량 바르면 좋습니다. 연구에 따르면 '모발에 수분을 공급하는 제품 사용이 건강한 모발 유지에 도움이 된다'고 합니다.",
        "머리카락에 텍스처를 추가하기 위해, 가벼운 스프레이 제품을 사용하여 스타일링하면 더욱 생동감 있는 모습을 연출할 수 있습니다."
      ]
    },
    "Skin": {
      "status": "피부는 전반적으로 깨끗하고 건강해 보이며, 수분감이 적절하나 약간의 유분이 느껴지는 상태입니다. 결점이 적고 자연스러운 광택이 나타나지만, 더욱 매끄럽고 균일한 톤을 위해 추가적인 관리가 필요할 수 있습니다.",
      "improvement_tips": [
        "하루에 두 번 수분 크림을 사용하여 피부의 수분을 유지하는 것이 좋습니다. '피부 수분 증가는 피부 장벽을 강화하는 데 필수적'이라고 합니다.",
        "주 1회 각질 제거를 통해 피부 결을 매끄럽게 하고, 자연스러운 광택을 더욱 부각시키는 것도 좋은 

In [10]:
retrieved_docs = retriever.invoke(query)
f = open('ref', 'w')
print("📚 참고 문서:")
f.write("📚 참고 문서:")
print("=" * 50)
f.write("=" * 50 + "\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[문서 {i}]")
    f.write(f"\n[문서 {i}]\n")
    if doc.metadata['source']:
        print(f"출처: {doc.metadata['source']}")
        f.write(f"출처: {doc.metadata['source']}\n")
    else:
        print(doc.page_content)
        f.write(doc.page_content + "\n")
    print("-" * 50)
    f.write("-" * 50 + "\n")
f.close()

📚 참고 문서:

[문서 1]
출처: data/docs/contour/셀럽 분석-얼굴 비율 등으로 어떤게 미인인지 연구(ㅇㄱ)-PubMed_Central.pdf
--------------------------------------------------

[문서 2]
출처: data/docs/contour/셀럽 분석-얼굴 비율 등으로 어떤게 미인인지 연구(ㅇㄱ)-PubMed_Central.pdf
--------------------------------------------------

[문서 3]
출처: data/docs/contour/셀럽 분석-얼굴 비율 등으로 어떤게 미인인지 연구(ㅇㄱ)-PubMed_Central.pdf
--------------------------------------------------

[문서 4]
출처: data/docs/hair/성격 인상 정보가 얼굴 매력 판단에 미치는 영향.pdf
--------------------------------------------------

[문서 5]
출처: data/docs/hair/성격 인상 정보가 얼굴 매력 판단에 미치는 영향.pdf
--------------------------------------------------

[문서 6]
출처: data/docs/hair/성격 인상 정보가 얼굴 매력 판단에 미치는 영향.pdf
--------------------------------------------------

[문서 7]
출처: data/docs/contour/셀럽 분석-얼굴 비율 등으로 어떤게 미인인지 연구(ㅇㄱ)-PubMed_Central.pdf
--------------------------------------------------

[문서 8]
출처: data/docs/contour/셀럽 분석-얼굴 비율 등으로 어떤게 미인인지 연구(ㅇㄱ)-PubMed_Central.pdf
--------------------------------------------------

[문서

# 마지막 질문에 대한 참고 문서 확인
retrieved_docs = retriever.invoke(query)

print("📚 참고 문서:")
print("=" * 50)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[문서 {i}]")
    print(doc.page_content)
    print("-" * 50)

In [11]:
## 사용 방법 요약

### 하이브리드 접근 (비용 효율적!) ⭐
- **임베딩**: HuggingFace multilingual-e5-large (무료, GPU/CPU, 다국어)
- **LLM**: OpenAI GPT-4o-mini (유료, 고품질, 비전 지원)

### 처음 실행할 때:
1. **.env 파일에 OpenAI API 키 설정** (OPENAI_API_KEY=sk-...)
2. **GPU 사용 가능 여부 확인** (없으면 Cell 4의 'cuda'를 'cpu'로 변경)
3. 셀 1-4를 순서대로 실행 (HuggingFace 임베딩 생성) ⭐ **무료!**
4. 셀 6을 실행 (RAG 체인 설정)
5. 셀 7을 실행 (텍스트만) 또는 셀 8을 실행 (이미지 포함) ⚠️ **OpenAI API 비용 발생**

### 다음번에 실행할 때 (임베딩 재사용):
1. 셀 1-2를 실행 (라이브러리 및 함수 로드)
2. **셀 5를 실행** (기존 벡터스토어 로드) ⭐ **4번 대신 5번!**
3. 셀 6을 실행 (RAG 체인 설정)
4. 셀 7 (텍스트만) 또는 셀 8 (이미지 포함) 실행

### 비용 및 성능 팁:
- **임베딩 (HuggingFace)**: 완전 무료! GPU 또는 CPU에서 실행
- **다국어 지원**: 한글 쿼리로 영어 논문도 검색 가능
- **텍스트 질의 (OpenAI)**: GPT-4o-mini API 비용만 발생, 매우 저렴
- **이미지 질의 (OpenAI)**: GPT-4o-mini 비전 모델, 이미지 분석 가능
- **한글 답변**: 프롬프트에 한글 강제 지시문 추가됨
- **논문 인용**: 참고 문서의 논문 내용을 직접 인용하여 전문성 향상
- 셀 7 또는 8의 `query` 변수만 수정해서 여러 질문을 테스트하세요.

### GPU vs CPU:
- **GPU 있음**: Cell 4와 5의 `device: 'cuda'` 그대로 사용 (빠름)
- **GPU 없음**: Cell 4와 5의 `device: 'cuda'`를 `device: 'cpu'`로 변경 (느리지만 작동)

### 기존 임베딩에서 마이그레이션:
- 기존에 `./chroma_db_hybrid` 또는 `./chroma_db_openai` 디렉토리가 있다면, 셀 4를 실행해서 새로운 HuggingFace 임베딩을 생성하세요.
- 새로운 임베딩은 `./chroma_db_e5` 디렉토리에 저장됩니다.

SyntaxError: invalid decimal literal (4111967523.py, line 5)

## 사용 방법 요약

### 하이브리드 접근 (비용 효율적!) ⭐
- **임베딩**: Ollama llama3 (무료, 로컬)
- **LLM**: OpenAI GPT-4o (유료, 고품질, 비전 지원)

### 처음 실행할 때:
1. **.env 파일에 OpenAI API 키 설정** (OPENAI_API_KEY=sk-...)
2. **Ollama llama3 모델 설치 확인** (터미널에서 `ollama pull llama3`)
3. 셀 1-4를 순서대로 실행 (Ollama 임베딩 생성) ⭐ **무료!**
4. 셀 6을 실행 (RAG 체인 설정)
5. 셀 7을 실행 (텍스트만) 또는 셀 8을 실행 (이미지 포함) ⚠️ **OpenAI API 비용 발생**

### 다음번에 실행할 때 (임베딩 재사용):
1. 셀 1-2를 실행 (라이브러리 및 함수 로드)
2. **셀 5를 실행** (기존 벡터스토어 로드) ⭐ **4번 대신 5번!**
3. 셀 6을 실행 (RAG 체인 설정)
4. 셀 7 (텍스트만) 또는 셀 8 (이미지 포함) 실행

### 비용 및 성능 팁:
- **임베딩 (Ollama)**: 완전 무료! 로컬에서 실행됨
- **텍스트 질의 (OpenAI)**: GPT-4o API 비용만 발생, 비교적 저렴
- **이미지 질의 (OpenAI)**: GPT-4o 비전 모델, 비용이 더 높지만 이미지 분석 가능
- **한글 답변**: 프롬프트에 한글 강제 지시문 추가됨
- 셀 7 또는 8의 `query` 변수만 수정해서 여러 질문을 테스트하세요.

### 기존 OpenAI 임베딩에서 마이그레이션:
- 기존에 `./chroma_db_openai` 디렉토리가 있다면, 셀 4를 실행해서 새로운 Ollama 임베딩을 생성하세요.
- 새로운 임베딩은 `./chroma_db_hybrid` 디렉토리에 저장됩니다.